In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Deploying the Model into Production with Merlin Systems and Triton Inference Server

At this point, when you reach out to this notebook, we expect that you have already executed the first notebook `01-Building-Recommender-Systems-PoC.ipynb` and exported all the required files and models. 

We are going to generate recommended items for a given user query (user_id) by following the steps described in the figure below.

![tritonensemble](../images/triton_ensemble.png)

Merlin Systems library have the set of operators to be able to serve multi-stage recommender systems built with Tensorflow on [Triton Inference Server](https://github.com/triton-inference-server/server)(TIS) easily and efficiently. Below, we will go through these operators and demonstrate their usage in serving a multi-stage system on Triton.

### Import required libraries and functions

In [3]:
import os
import numpy as np
import feast
import faiss
import pandas as pd

from nvtabular import ColumnSchema, Schema

from merlin.systems.dag.ensemble import Ensemble
from merlin.systems.dag.ops.session_filter import FilterCandidates
from merlin.systems.dag.ops.softmax_sampling import SoftmaxSampling
from merlin.systems.dag.ops.tensorflow import PredictTensorflow
from merlin.systems.dag.ops.unroll_features import UnrollFeatures
from merlin.systems.triton.utils import run_triton_server, run_ensemble_on_tritonserver

### Register our features

We have defined our user and item features definitions in the `user_features.py` and  `item_features.py` files. With FeatureView() users can register data sources in their organizations into Feast, and then use those data sources for both training and online inference. In the `user_features.py` and `item_features.py` files, we are telling Feast where to find user and item features.

Before we move on to the next steps, we need to perform `feast apply`command as directed below.  With that, we register our features, we can apply the changes to create our feature registry and store all entity and feature view definitions in a local SQLite online store called `online_store.db`.

In [4]:
!cd /Merlin/examples/Deploying-multi-stage-RecSys/feature_repo
!feast apply

/usr/local/lib/python3.8/dist-packages/feast/feature_view.py:100: DeprecationWarning: The argument 'input' is being deprecated. Please use 'batch_source' instead. Feast 0.13 and onwards will not support the argument 'input'.
  warnings.warn(
Created data source /Merlin/examples/Deploying-multi-stage-RecSys/feature_repo/data/item_features.parquet
Created data source /Merlin/examples/Deploying-multi-stage-RecSys/feature_repo/data/user_features.parquet
Created entity user_id
Created entity item_id
Created feature view item_features
Created feature view user_features

Created sqlite table feature_repo_item_features
Created sqlite table feature_repo_user_features



### Feast Materialize

After we execute `apply` and registered our features and created our online local store, now we need to perform [materialization](https://docs.feast.dev/how-to-guides/running-feast-in-production) operation. This is done to keep our online store up to date and get it ready for prediction. For that we need to run a job that loads feature data from our feature view sources into our online store. As we add new features to our offline stores, we can continuously materialize them to keep our online store up to date by finding the latest feature values for each user. 

When you run the `feast materialize ..` command below, you will see a message <i>Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store </i>  printed out below

Note that materialization step takes some time.. 

In [6]:
!feast materialize 1995-01-01T01:01:01 2025-01-01T01:01:01

Materializing 2 feature views from 1995-01-01 01:01:01+00:00 to 2025-01-01 01:01:01+00:00 into the sqlite online store.

item_features:
100%|███████████████████████████████████████████████████| 3078306/3078306 [14:53<00:00, 3447.06it/s]
user_features:
100%|█████████████████████████████████████████████████████| 294736/294736 [03:18<00:00, 1488.17it/s]


Now, let's check our feature_repo structure again after we ran `apply` and `materialize` commands.

In [4]:
# set up the base dir to for feature store
BASE_DIR = os.environ.get("BASE_DIR", "/Merlin/examples/Deploying-multi-stage-RecSys/")
feature_repo_path = os.path.join(BASE_DIR, 'feature_repo')
!tree $feature_repo_path

/Merlin/examples/Deploying-multi-stage-RecSys/feature_repo
├── __init__.py
├── data
│   ├── item_features.parquet
│   ├── online_store.db
│   ├── registry.db
│   └── user_features.parquet
├── feature_store.yaml
├── item_features.py
└── user_features.py

1 directory, 8 files


We use `configure_tensorflow` function to prevent the Tensorflow to claim entire GPU memory. With this func, we let TF to allocate 50% of the available GPU memory. 

In [5]:
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()

03/30/2022 10:48:55 PM INFO:init


<function tensorflow.python.dlpack.dlpack.from_dlpack(dlcapsule)>

Create a folder for faiss index path

In [9]:
if not os.path.isdir(os.path.join(BASE_DIR + 'faiss_index')):
    os.makedirs(os.path.join(BASE_DIR + 'faiss_index'))

Define paths for ranking model, retrieval model, feast feature repo and faiss index path

In [15]:
faiss_index_path = BASE_DIR + 'faiss_index' + "/index.faiss"
feast_repo_path = BASE_DIR + "feature_repo/"
retrieval_model_path = BASE_DIR + "query_tower/"
ranking_model_path = BASE_DIR + "dlrm/"

Create a request schema that we are going to use when sending a request to Triton Infrence Server (TIS).

In [16]:
request_schema = Schema(
    [
        ColumnSchema("user_id", dtype=np.int32),
    ]
)

### Set up Faiss

`QueryFaiss` operator creates an interface between a FAISS Approximate Nearest Neighbors (ANN) Index and Triton Infrence Server. For a given input query vector, we do an ANN search query to find the ids of top-k nearby nodes in the index. 

`QueryFeast` operator is responsible for ensuring that our feast feature store can communicate correctly with tritonserver for the ensemble feast feature look ups.

In [17]:
from merlin.systems.dag.ops.faiss import QueryFaiss, setup_faiss 
from merlin.systems.dag.ops.feast import QueryFeast 

item_embeddings = np.ascontiguousarray(
    pd.read_parquet(BASE_DIR + "item_embeddings.parquet").to_numpy()
)

`setup_faiss` is  a utility function that will create a Faiss index from an embedding vector with using L2 distance.

In [18]:
feature_store = feast.FeatureStore(feast_repo_path)
setup_faiss(item_embeddings, faiss_index_path)

Fetch user features with `QueryFeast` operator from the feature store.

In [19]:
user_features = ["user_id"] >> QueryFeast.from_feature_view(
    store=feature_store,
    path=feast_repo_path,
    view="user_features",
    column="user_id",
    include_id=True,
)

Retrieve top-K candidate items using `retrieval model` that are relevant for a given user. We use `PredictTensorflow()` operator that takes a tensorflow model and packages it correctly for TIS to run with the tensorflow backend.

In [20]:
retrieval = (
    user_features
    >> PredictTensorflow(retrieval_model_path)
    >> QueryFaiss(faiss_index_path, topk=100)
)

2022-03-30 22:55:32.999743: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 22:55:34.169618: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


03/30/2022 10:55:36 PM WARNING:No training configuration found in save file, so the model was *not* compiled. Compile it manually.


Fetch item features for the candidate items that are retrieved from the retrieval step above from the feature store.

In [21]:
item_features = retrieval["candidate_ids"] >> QueryFeast.from_feature_view(
    store=feature_store,
    path=feast_repo_path,
    view="item_features",
    column="candidate_ids",
    output_prefix="item",
    include_id=True,
)

Merge the user features and items features to create the all set of combined features that were used in model training using `UnrollFeatures` operator which takes a target column and joins the "unroll" columns to the target. This helps when broadcasting a series of user features to a set of items.

In [22]:
user_features_to_unroll = [
    "user_id",
    "user_shops",
    "user_profile",
    "user_group",
    "user_gender",
    "user_age",
    "user_consumption_2",
    "user_is_occupied",
    "user_geography",
    "user_intentions",
    "user_brands",
    "user_categories",
]

combined_features = item_features >> UnrollFeatures(
    "item_id", user_features[user_features_to_unroll]
)

Rank the combined features using the trained ranking model, which is a DLRM model for this example. We feed the path of the ranking model to `PredictTensorflow()` operator.

In [23]:
ranking = combined_features >> PredictTensorflow(ranking_model_path)

For the ordering we use `SoftmaxSampling()` operator. This operator sorts all inputs in descending order given the input ids and prediction introducing some randomization into the ordering by sampling items from the softmax of the predicted relevance scores, and finally returns top-k ordered items.

In [24]:
ordering = combined_features["item_id"] >> SoftmaxSampling(
    relevance_col=ranking["output_1"], topk=10, temperature=20.0
)

### Export Graph as Ensemble
The last step is to create the ensemble artifacts that TIS can consume. To make these artifacts import the Ensemble class. This class  represents an entire ensemble consisting of multiple models that run sequentially in TIS initiated by an inference request. It is responsible with interpreting the graph and exporting the correct files for TIS.

When we create an Ensemble object we feed the graph and a schema representing the starting input of the graph.  After we create the ensemble object, we export the graph, supplying an export path for the `ensemble.export()` function. This returns an ensemble config which represents the entire inference pipeline and a list of node-specific configs.

Create the folder to export the models and config files.

In [25]:
if not os.path.isdir(os.path.join(BASE_DIR + 'poc_ensemble')):
    os.makedirs(os.path.join(BASE_DIR + 'poc_ensemble'))

In [27]:
# define the path where all the models and config files exported to
export_path = os.path.join(BASE_DIR + 'poc_ensemble')

ensemble = Ensemble(ordering, request_schema)
ens_config, node_configs = ensemble.export(export_path)

Let's check our export_path structure

In [28]:
!tree $export_path

/Merlin/examples/Deploying-multi-stage-RecSys/poc_ensemble
├── 0_queryfeast
│   ├── 1
│   │   └── model.py
│   └── config.pbtxt
├── 1_predicttensorflow
│   ├── 1
│   │   └── model.savedmodel
│   │       ├── assets
│   │       ├── keras_metadata.pb
│   │       ├── saved_model.pb
│   │       └── variables
│   │           ├── variables.data-00000-of-00001
│   │           └── variables.index
│   └── config.pbtxt
├── 2_queryfaiss
│   ├── 1
│   │   ├── index.faiss
│   │   │   └── index.faiss
│   │   └── model.py
│   └── config.pbtxt
├── 3_queryfeast
│   ├── 1
│   │   └── model.py
│   └── config.pbtxt
├── 4_unrollfeatures
│   ├── 1
│   │   └── model.py
│   └── config.pbtxt
├── 5_predicttensorflow
│   ├── 1
│   │   └── model.savedmodel
│   │       ├── assets
│   │       ├── keras_metadata.pb
│   │       ├── saved_model.pb
│   │       └── variables
│   │           ├── variables.data-00000-of-00001
│   │           └── variables.index
│   └── config.pbtxt
├── 6_softmaxsampling
│   ├── 1
│   │   └

### Retrieving Recommendations from Triton

It is time to deploy the all the models as an ensemble model to Triton Inference very easily using Merlin Systems library. Now we can launch our triton server and load our models, and get a response for our query with a utility function `run_ensemble_on_tritonserver()`.

In [29]:
# create a request to be sent to TIS
from merlin.core.dispatch import make_df

request = make_df({"user_id": [1]})
request["user_id"] = request["user_id"].astype(np.int32)

response = run_ensemble_on_tritonserver(
    export_path, ensemble.graph.output_schema.column_names, request, "ensemble_model"
)

I0330 22:57:40.613079 11012 tensorflow.cc:2176] TRITONBACKEND_Initialize: tensorflow
I0330 22:57:40.613268 11012 tensorflow.cc:2186] Triton TRITONBACKEND API version: 1.8
I0330 22:57:40.613283 11012 tensorflow.cc:2192] 'tensorflow' TRITONBACKEND API version: 1.8
I0330 22:57:40.613299 11012 tensorflow.cc:2216] backend configuration:
{"cmdline":{"version":"2"}}
I0330 22:57:40.775601 11012 pinned_memory_manager.cc:240] Pinned memory pool is created at '0x7faec4000000' with size 268435456
I0330 22:57:40.775981 11012 cuda_memory_manager.cc:105] CUDA memory pool is created on device 0 with size 67108864
I0330 22:57:40.781158 11012 model_repository_manager.cc:994] loading: 0_queryfeast:1
I0330 22:57:40.881596 11012 model_repository_manager.cc:994] loading: 1_predicttensorflow:1
I0330 22:57:40.889764 11012 backend.cc:46] TRITONBACKEND_Initialize: nvtabular
I0330 22:57:40.889792 11012 backend.cc:53] Triton TRITONBACKEND API version: 1.8
I0330 22:57:40.889805 11012 backend.cc:56] 'nvtabular' TRI

Signal (2) received.


I0330 22:57:55.632500 11012 server.cc:252] Waiting for in-flight requests to complete.
I0330 22:57:55.632531 11012 model_repository_manager.cc:1026] unloading: ensemble_model:1
I0330 22:57:55.632647 11012 model_repository_manager.cc:1026] unloading: 6_softmaxsampling:1
I0330 22:57:55.632770 11012 model_repository_manager.cc:1026] unloading: 5_predicttensorflow:1
I0330 22:57:55.632872 11012 model_repository_manager.cc:1132] successfully unloaded 'ensemble_model' version 1
I0330 22:57:55.632954 11012 model_repository_manager.cc:1026] unloading: 4_unrollfeatures:1
I0330 22:57:55.633005 11012 model_repository_manager.cc:1026] unloading: 3_queryfeast:1
I0330 22:57:55.633000 11012 backend.cc:160] TRITONBACKEND_ModelInstanceFinalize: delete instance state
Signal (11) received.
I0330 22:57:55.633048 11012 model_repository_manager.cc:1026] unloading: 2_queryfaiss:1
I0330 22:57:55.633214 11012 backend.cc:160] TRITONBACKEND_ModelInstanceFinalize: delete instance stateI0330 22:57:55.633231 11012 t

Convert our response to a numpy array and print it out.

In [30]:
output= response.as_numpy('ordered_ids')
output

array([[1418158],
       [ 861350],
       [1562428],
       [1979510],
       [ 795631],
       [1339132],
       [ 729802],
       [1168971],
       [2988914],
       [1357524]], dtype=int32)

Note that these item ids are encoded values, not the raw original values. We will eventually create the reverse dictionary lookup functionality to be able to map these encoded item ids to their original raw ids with one-line of code. But if you really want to do it now, you can easily map these ids to their original values using the `unique.item_id.parquet` file stored in the `categories` folder.

That's it! You finished deploying a multi-stage Recommender Systems on Triton Inference Server using Merlin framework.